<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Practice/Practicing_text_generation_with_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
len(shakespeare_text)

1115394

In [4]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

# total number of distinct characters
max_id = len(tokenizer.word_index)
# total number of characters
dataset_size = tokenizer.document_count

In [5]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [7]:
print(max_id)
print(dataset_size)

39
1115394


In [8]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [9]:
encoded.shape

(1115394,)

In [10]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [12]:
next(iter(dataset))

<tf.Tensor: shape=(), dtype=int64, numpy=19>

In [13]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [14]:
next(iter(dataset))

<_VariantDataset shapes: (), types: tf.int64>

In [15]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset

<FlatMapDataset shapes: (None,), types: tf.int64>

In [16]:
next(iter(dataset))

<tf.Tensor: shape=(101,), dtype=int64, numpy=
array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 21,  1,
       19,  3,  8,  1,  0, 16,  1,  0, 22,  8,  3, 18,  1,  1, 12,  0,  4,
        9, 15,  0, 19, 13,  8,  2,  6,  1,  8, 17,  0,  6,  1,  4,  8,  0,
       14,  1,  0,  7, 22,  1,  4, 24, 26, 10, 10,  4, 11, 11, 23, 10,  7,
       22,  1,  4, 24, 17,  0,  7, 22,  1,  4, 24, 26, 10, 10, 19,  5,  8,
        7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 15,  3, 13,  0])>

In [17]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda window: (window[:, :-1], window[:, 1:]))

In [18]:
next(iter(dataset))

(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
 array([[ 6,  4, 25, ...,  0, 22,  3],
        [ 0,  4,  0, ..., 24,  0,  2],
        [ 0, 15,  3, ..., 20,  1,  1],
        ...,
        [ 0,  3, 19, ...,  0,  3,  9],
        [ 0, 19,  3, ...,  3,  7,  1],
        [ 0, 11,  5, ...,  1,  5,  8]])>,
 <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
 array([[ 4, 25,  1, ..., 22,  3,  3],
        [ 4,  0, 20, ...,  0,  2,  6],
        [15,  3, 13, ...,  1,  1,  7],
        ...,
        [ 3, 19,  0, ...,  3,  9,  1],
        [19,  3, 11, ...,  7,  1, 17],
        [11,  5, 24, ...,  5,  8,  0]])>)

In [19]:
dataset = dataset.map(lambda x_batch, y_batch : (tf.one_hot(x_batch, depth=max_id), y_batch))

In [20]:
next(iter(dataset))

(<tf.Tensor: shape=(32, 100, 39), dtype=float32, numpy=
 array([[[0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..

In [22]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [23]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

history = model.fit(dataset, epochs=20)

Epoch 1/20
31368/31368 [==============================] - 414s 13ms/step - loss: 0.9639
Epoch 2/20
31368/31368 [==============================] - 420s 13ms/step - loss: 0.9671
Epoch 3/20
31368/31368 [==============================] - 420s 13ms/step - loss: 1.0264
Epoch 4/20
  273/31368 [..............................] - ETA: 6:51 - loss: 1.2174

KeyboardInterrupt: ignored